<a href="https://colab.research.google.com/github/shahad-jeza/Saudi_medical_AI_lab_EX/blob/main/pdb_file_reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# reading trajectories
a trajectory is a "snapshot" frames of molecular postions over time, here we have 5

Atom >> defines the data for each atom, like x,y,z coords

MODEL and ENDMDL >>> defines the start and end of each frame

In [1]:
!pip install MDAnalysis

In [2]:
import MDAnalysis as mda

# Universe: the centeral data structure, contains all atoms and trajec
u = mda.Universe("/content/6y0f_onco_prod_cent_frame.pdb")

/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/PDBParser.py:346: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn(wmsg)


In [3]:
# take a look at the atoms
for a in (u.atoms):
  print(a)

Streaming output truncated to the last 5000 lines.
<Atom 6641: C of type C of resname GLU, resid 446 and segid A and altLoc >
<Atom 6642: O of type O of resname GLU, resid 446 and segid A and altLoc >
<Atom 6643: N of type N of resname ARG, resid 447 and segid A and altLoc >
<Atom 6644: H of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6645: CA of type C of resname ARG, resid 447 and segid A and altLoc >
<Atom 6646: HA of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6647: CB of type C of resname ARG, resid 447 and segid A and altLoc >
<Atom 6648: HB1 of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6649: HB2 of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6650: CG of type C of resname ARG, resid 447 and segid A and altLoc >
<Atom 6651: HG1 of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6652: HG2 of type H of resname ARG, resid 447 and segid A and altLoc >
<Atom 6653: CD of type C of resname A

In [4]:
# MDAnalysis only accsess one trajec at a time which helps when having large set of frames
for ts in u.trajectory:
  print(f"Frame: {ts.frame}")
  print(f"number of atomes: {u.atoms.n_atoms}")
  print(f"coord of first atom: {u.atoms[0].position}")

Frame: 0
number of atomes: 11640
coord of first atom: [26.218 50.462 44.734]
Frame: 1
number of atomes: 11640
coord of first atom: [43.848 42.251 53.071]
Frame: 2
number of atomes: 11640
coord of first atom: [81.521 42.669 42.189]
Frame: 3
number of atomes: 11640
coord of first atom: [83.574 56.421 23.507]
Frame: 4
number of atomes: 11640
coord of first atom: [87.746 54.331 51.928]
Frame: 5
number of atomes: 11640
coord of first atom: [91.569 60.909 46.425]


the number of atoms stayes constant accross trajectories only the coords change



---



In [27]:
# what is the distance between atom 1 and 2 in each traj

atom1 = u.atoms[0]  # First atom
atom2 = u.atoms[1]  # Second atom

for ts in u.trajectory:
    distance = atom1.position - atom2.position
    print(f"Frame {ts.frame}: Distance = {distance}")

Frame 0: Distance = [-0.34000015  0.23000336 -0.91999817]
Frame 1: Distance = [ 0.13000107 -0.36999893 -0.9300003 ]
Frame 2: Distance = [0.25       0.66999817 0.7099991 ]
Frame 3: Distance = [ 0.1499939  -0.77000046  0.6399994 ]
Frame 4: Distance = [-0.00999451  1.0100021   0.05000305]
Frame 5: Distance = [-0.55999756 -0.829998   -0.11999893]


In [29]:
from MDAnalysis.lib.distances import calc_angles
#calculate angles between atoms

atom3 = u.atoms[2]  # Third atom
for ts in u.trajectory:
    angle = calc_angles(atom1.position, atom2.position, atom3.position)
    print(f"Frame {ts.frame}: Angle = {angle}")

Frame 0: Angle = 0.5969837366754257
Frame 1: Angle = 0.6493369916638598
Frame 2: Angle = 0.6360635335920618
Frame 3: Angle = 0.6025433924379953
Frame 4: Angle = 0.5981238308535288
Frame 5: Angle = 0.6426475242395385


the RMSD calculates the average deviation between two configurations of atomic coordinates. It’s commonly used to assess how much two molecular structures differ,

In [31]:
from MDAnalysis.analysis.rms import RMSD

rmsd = RMSD(u, u, select="backbone")
rmsd.run()
print(rmsd.results.rmsd)

/usr/local/lib/python3.10/dist-packages/MDAnalysis/analysis/base.py:522: UserWarning: Reader has no dt information, set to 1.0 ps
  self.times[idx] = ts.time
/usr/local/lib/python3.10/dist-packages/MDAnalysis/coordinates/base.py:757: UserWarning: Reader has no dt information, set to 1.0 ps
  return self.ts.time


[[0.00000000e+00 0.00000000e+00 1.88505526e-06]
 [1.00000000e+00 1.00000000e+00 3.04365382e+00]
 [2.00000000e+00 2.00000000e+00 3.01927065e+00]
 [3.00000000e+00 3.00000000e+00 2.69981190e+00]
 [4.00000000e+00 4.00000000e+00 3.05563883e+00]
 [5.00000000e+00 5.00000000e+00 3.26558737e+00]]




---



Another way i found is to read it as a pandas dataframe

In [34]:
!pip install biopandas prody

In [35]:
import pandas as pd
from biopandas.pdb import PandasPdb
from prody import parsePDBHeader
from typing import Optional

def read_pdb_to_dataframe(
    pdb_path: Optional[str] = None,
    model_index: int = 1,
    parse_header: bool = True,
    ) -> pd.DataFrame:
    """
    Read a PDB file, and return a Pandas DataFrame containing the atomic coordinates and metadata.

    Args:
        pdb_path (str, optional): Path to a local PDB file to read. Defaults to None.
        model_index (int, optional): Index of the model to extract from the PDB file, in case
            it contains multiple models. Defaults to 1.
        parse_header (bool, optional): Whether to parse the PDB header and extract metadata.
            Defaults to True.

    Returns:
        pd.DataFrame: A DataFrame containing the atomic coordinates and metadata, with one row
            per atom
    """
    atomic_df = PandasPdb().read_pdb(pdb_path)
    if parse_header:
        header = parsePDBHeader(pdb_path)
    else:
        header = None
    atomic_df = atomic_df.get_model(model_index)
    if len(atomic_df.df["ATOM"]) == 0:
        raise ValueError(f"No model found for index: {model_index}")

    return pd.concat([atomic_df.df["ATOM"], atomic_df.df["HETATM"]]), header

In [36]:
df, df_header = read_pdb_to_dataframe('/content/6y0f_onco_prod_cent_frame.pdb')
df.head(10)

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx,model_id
0,ATOM,1,,N,,ASN,,A,37,,...,50.462,44.734,1.0,0.0,,,N,NaN,5,1
1,ATOM,2,,H1,,ASN,,A,37,,...,50.232,45.654,1.0,0.0,,,H,NaN,6,1
2,ATOM,3,,H2,,ASN,,A,37,,...,51.112,44.774,1.0,0.0,,,H,NaN,7,1
3,ATOM,4,,H3,,ASN,,A,37,,...,49.642,44.254,1.0,0.0,,,H,NaN,8,1
4,ATOM,5,,CA,,ASN,,A,37,,...,51.142,44.024,1.0,0.0,,,C,NaN,9,1
5,ATOM,6,,HA,,ASN,,A,37,,...,51.562,43.144,1.0,0.0,,,H,NaN,10,1
6,ATOM,7,,CB,,ASN,,A,37,,...,52.312,44.944,1.0,0.0,,,C,NaN,11,1
7,ATOM,8,,HB1,,ASN,,A,37,,...,51.842,45.664,1.0,0.0,,,H,NaN,12,1
8,ATOM,9,,HB2,,ASN,,A,37,,...,52.782,45.524,1.0,0.0,,,H,NaN,13,1
9,ATOM,10,,CG,,ASN,,A,37,,...,53.392,44.184,1.0,0.0,,,C,NaN,14,1


In [37]:
df.loc[df['atom_name']=='CA', ['x_coord', 'y_coord', 'z_coord']] # show coors for carboon atoms

,x_coord,y_coord,z_coord
4,27.358,51.142,44.024
18,30.178,49.702,41.874
32,34.118,50.112,41.964
49,35.158,52.582,39.094
73,38.298,53.662,37.134
...,...,...,...
11509,46.728,57.772,23.134
11531,48.298,58.192,19.684
11548,51.878,57.682,21.154
11558,51.028,54.572,23.314
